Ceaser cipher

In [ ]:
def caesar_encrypt(text, shift):
    result = ""
    for char in text:
        if char.isalpha():
            base = ord('A') if char.isupper() else ord('a')
            result += chr((ord(char) - base + shift) % 26 + base)
        else:
            result += char
    return result

def caesar_decrypt(cipher, shift):
    return caesar_encrypt(cipher, -shift)

# Example
plaintext = "HELLO WORLD"
shift = 3
cipher = caesar_encrypt(plaintext, shift)
print("Caesar Encrypted:", cipher)
print("Caesar Decrypted:", caesar_decrypt(cipher, shift))


Caesar Encrypted: KHOOR ZRUOG
Caesar Decrypted: HELLO WORLD



Play fair

In [ ]:
def generate_key_matrix(key):
    key = key.upper().replace("J", "I")
    matrix, used = [], set()
    for c in key + "ABCDEFGHIKLMNOPQRSTUVWXYZ":
        if c.isalpha() and c not in used:
            matrix.append(c)
            used.add(c)
    return [matrix[i:i+5] for i in range(0, 25, 5)]
def find_position(matrix, ch):
    for i in range(len(matrix)):
        row = matrix[i]
        if ch in row:
            return i, row.index(ch)
def playfair_encrypt(text, key):
    matrix = generate_key_matrix(key)
    text = "".join([c for c in text.upper().replace("J","I") if c.isalpha()])
    pairs, i = [], 0
    while i < len(text):
        a, b = text[i], text[i+1] if i+1 < len(text) else "X"
        if a == b:
            pairs.append((a,"X"))
            i += 1
        else:
            pairs.append((a,b))
            i += 2
    result = ""
    for a, b in pairs:
        r1,c1 = find_position(matrix,a)
        r2,c2 = find_position(matrix,b)
        if r1==r2:
            result += matrix[r1][(c1+1)%5]+matrix[r2][(c2+1)%5]
        elif c1==c2:
            result += matrix[(r1+1)%5][c1]+matrix[(r2+1)%5][c2]
        else:
            result += matrix[r1][c2]+matrix[r2][c1]
    return result
def playfair_decrypt(cipher, key):
    matrix = generate_key_matrix(key)
    result = ""
    for i in range(0, len(cipher), 2):
        a,b = cipher[i], cipher[i+1]
        r1,c1 = find_position(matrix,a)
        r2,c2 = find_position(matrix,b)
        if r1==r2:
            result += matrix[r1][(c1-1)%5]+matrix[r2][(c2-1)%5]
        elif c1==c2:
            result += matrix[(r1-1)%5][c1]+matrix[(r2-1)%5][c2]
        else:
            result += matrix[r1][c2]+matrix[r2][c1]
    cleaned = ""
    for i in range(len(result)):
        ch = result[i]
        if not (ch=="X" and 0<i<len(result)-1 and result[i-1]==result[i+1]):
            cleaned += ch
    if cleaned.endswith("X"):
        cleaned = cleaned[:-1]
    return cleaned
key = "PLAYFAIR"
plaintext = "SRIYA"
cipher = playfair_encrypt(plaintext, key)
print("Encrypted:", cipher)
print("Decrypted:", playfair_decrypt(cipher, key))


Encrypted: OCCPYW
Decrypted: SRIYA


Hill cipher

In [ ]:
import numpy as np

def hill_encrypt(plaintext, key_matrix):
    n = len(key_matrix)
    plaintext = plaintext.upper()

    # Padding with X if not multiple of n
    if len(plaintext) % n != 0:
        plaintext += "X" * (n - (len(plaintext) % n))

    cipher = ""
    for i in range(0, len(plaintext), n):
        block = [ord(plaintext[j]) - 65 for j in range(i, i+n)]
        result = np.dot(key_matrix, block) % 26
        cipher += ''.join(chr(int(x) + 65) for x in result)
    return cipher

def hill_decrypt(cipher, key_matrix):
    n = len(key_matrix)
    det = round(np.linalg.det(key_matrix))% 26
    inv_det = pow(det,-1, 26)
    # adj = np.round(det * np.linalg.inv(key_matrix)) % 26
    # inv_matrix = (inv_det * adj) % 26
    inv_matrix=np.round(inv_det*det*np.linalg.inv(key_matrix))%26
    plaintext = ""
    for i in range(0, len(cipher), n):
        block = [ord(cipher[j]) - 65 for j in range(i, i+n)]
        result = np.dot(inv_matrix, block) % 26
        plaintext += ''.join(chr(int(x) + 65) for x in result)
    return plaintext

# Example
key_matrix = np.array([[3, 3, 1],
                       [2, 5, 0],
                       [0, 0, 1]])

plaintext = "HELLO"
cipher = hill_encrypt(plaintext, key_matrix)
print("Hill Encrypted:", cipher)
print("Hill Decrypted:", hill_decrypt(cipher, key_matrix))


Hill Encrypted: SILUOX
Hill Decrypted: HELLOX


Rail fence

In [ ]:
def encrypt(pt, key):
    rows = [''] * key
    row = 0
    step = 1
    for ch in pt:
        rows[row] += ch
        if row == 0:
            step = 1
        elif row == key - 1:
            step = -1
        row += step
    return rows, ''.join(rows)


def decrypt(cipher, key):
    # Step 1: Determine the zig-zag pattern of positions
    n = len(cipher)
    pos = [[] for _ in range(key)]
    row, step = 0, 1
    for i in range(n):
        pos[row].append(i)
        if row == 0:
            step = 1
        elif row == key - 1:
            step = -1
        row += step

    # Step 2: Fill rows with characters from cipher
    idx = 0
    rows = [''] * key
    for r in range(key):
        for _ in pos[r]:
            rows[r] += cipher[idx]
            idx += 1

    # Step 3: Rebuild plaintext using the zig-zag order
    ans = []
    row, step = 0, 1
    for i in range(n):
        ans.append(rows[row][0])
        rows[row] = rows[row][1:]  # consume character
        if row == 0:
            step = 1
        elif row == key - 1:
            step = -1
        row += step
    return ''.join(ans)


# Example
plaintext = "GEEKSFORGEEKS"
key = 3
rows, cipher = encrypt(plaintext, key)
print("Rail Fence Encrypted:", cipher)
print("Rail Fence Decrypted:", decrypt(cipher, key))


Rail Fence Encrypted: GSGSEKFREKEOE
Rail Fence Decrypted: GEEKSFORGEEKS


Row transformation

In [ ]:
import math
def encryptMessage(msg):
    col = len(key)
    row = math.ceil(len(msg) / col)
    padding = '_' * ((row * col) - len(msg))
    msg += padding
    matrix = [list(msg[i:i + col]) for i in range(0, len(msg), col)]
    key_order = sorted(key)
    cipher = ''
    for char in key_order:
        col_idx = key.index(char)
        cipher += ''.join(row[col_idx] for row in matrix)
    return cipher
def decryptMessage(cipher):
    col = len(key)
    row = math.ceil(len(cipher) / col)
    key_order = sorted(key)
    matrix = [[None] * col for _ in range(row)]
    idx = 0
    for char in key_order:
        col_idx = key.index(char)
        for r in range(row):
            matrix[r][col_idx] = cipher[idx]
            idx += 1
    msg = ''.join(char for row in matrix for char in row)
    return msg.rstrip('_')
msg=input("Enter plain text:")
key=input("Enter key with distinct characters:")
cipher = encryptMessage(msg)
print("Encrypted Message:", cipher)
print("Decrypted Message:", decryptMessage(cipher))


Enter plain text:abcdefghijkl
Enter key with distinct characters:rfna
Encrypted Message: dhlbfjcgkaei
Decrypted Message: abcdefghijkl


Column transformation

In [ ]:
import math
def encryptMessage(msg, key):
    col = len(key)
    row = math.ceil(len(msg) / col)
    padding = '_' * ((row * col) - len(msg))
    msg += padding
    matrix = [list(msg[i:i + col]) for i in range(0, len(msg), col)]
    key_order = sorted(list(key))
    cipher = ''
    for char in key_order:
        col_idx = key.index(char)
        cipher += ''.join(row[col_idx] for row in matrix)
    return cipher
def decryptMessage(cipher, key, strip_padding=False):
    col = len(key)
    row = math.ceil(len(cipher) / col)
    key_order = sorted(list(key))
    matrix = [[None] * col for _ in range(row)]
    idx = 0
    for char in key_order:
        col_idx = key.index(char)
        for r in range(row):
            if idx < len(cipher):
                matrix[r][col_idx] = cipher[idx]
                idx += 1
    msg = ''.join(char for row in matrix for char in row)
    if strip_padding:
        msg = msg.rstrip('_')
    return msg
msg = input("Enter plain text: ").replace(" ", "").upper()
key = input("Enter key with distinct characters: ").upper()
rounds = int(input("Enter number of rounds: "))
cipher = msg
for _ in range(rounds):
    cipher = encryptMessage(cipher, key)
decrypted = cipher
for i in range(rounds):
    decrypted = decryptMessage(decrypted, key, strip_padding=(i == rounds - 1))
print("Encrypted Message:", cipher)
print("Decrypted Message:", decrypted)



Enter plain text: fghbjnkm,
Enter key with distinct characters: qwerty
Enter number of rounds: 5
Encrypted Message: H,FJK_B_GMN_
Decrypted Message: FGHBJNKM,


DES

In [ ]:
def hex2bin(s):
    mp = {'0': "0000", '1': "0001", '2': "0010", '3': "0011",
          '4': "0100", '5': "0101", '6': "0110", '7': "0111",
          '8': "1000", '9': "1001", 'A': "1010", 'B': "1011",
          'C': "1100", 'D': "1101", 'E': "1110", 'F': "1111"}
    return ''.join(mp[ch] for ch in s)

def bin2hex(s):
    mp = {"0000": '0', "0001": '1', "0010": '2', "0011": '3',
          "0100": '4', "0101": '5', "0110": '6', "0111": '7',
          "1000": '8', "1001": '9', "1010": 'A', "1011": 'B',
          "1100": 'C', "1101": 'D', "1110": 'E', "1111": 'F'}
    return ''.join(mp[s[i:i+4]] for i in range(0, len(s), 4))

def permute(k, arr):
    return ''.join(k[i - 1] for i in arr)

def shift_left(k, n):
    return k[n:] + k[:n]

def xor(a, b):
    res=''
    for i,j in zip(a,b):
        if i!=j:
            res+='1'
        else:
            res+='0'
    return res

# Initial Permutation Table
initial_perm = [58, 50, 42, 34, 26, 18, 10, 2, 60, 52, 44, 36,
                28, 20, 12, 4, 62, 54, 46, 38, 30, 22, 14, 6,
                64, 56, 48, 40, 32, 24, 16, 8, 57, 49, 41, 33,
                25, 17, 9, 1, 59, 51, 43, 35, 27, 19, 11, 3,
                61, 53, 45, 37, 29, 21, 13, 5, 63, 55, 47, 39,
                31, 23, 15, 7]

# Expansion D-box Table
exp_d = [32, 1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 8, 9, 10, 11, 12, 13,
         12, 13, 14, 15, 16, 17, 16, 17, 18, 19, 20, 21, 20, 21,
         22, 23, 24, 25, 24, 25, 26, 27, 28, 29, 28, 29, 30, 31,
         32, 1]

# Final Permutation Table
final_perm = [40, 8, 48, 16, 56, 24, 64, 32, 39, 7, 47, 15, 55, 23,
              63, 31, 38, 6, 46, 14, 54, 22, 62, 30, 37, 5, 45, 13,
              53, 21, 61, 29, 36, 4, 44, 12, 52, 20, 60, 28, 35, 3,
              43, 11, 51, 19, 59, 27, 34, 2, 42, 10, 50, 18, 58, 26,
              33, 1, 41, 9, 49, 17, 57, 25]

# S-boxes
sbox = [
    [[14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7],
     [0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8],
     [4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0],
     [15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13]],

    [[15,1,8,14,6,11,3,4,9,7,2,13,12,0,5,10],
     [3,13,4,7,15,2,8,14,12,0,1,10,6,9,11,5],
     [0,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15],
     [13,8,10,1,3,15,4,2,11,6,7,12,0,5,14,9]],

    [[10,0,9,14,6,3,15,5,1,13,12,7,11,4,2,8],
     [13,7,0,9,3,4,6,10,2,8,5,14,12,11,15,1],
     [13,6,4,9,8,15,3,0,11,1,2,12,5,10,14,7],
     [1,10,13,0,6,9,8,7,4,15,14,3,11,5,2,12]],

    [[7,13,14,3,0,6,9,10,1,2,8,5,11,12,4,15],
     [13,8,11,5,6,15,0,3,4,7,2,12,1,10,14,9],
     [10,6,9,0,12,11,7,13,15,1,3,14,5,2,8,4],
     [3,15,0,6,10,1,13,8,9,4,5,11,12,7,2,14]],

    [[2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9],
     [14,11,2,12,4,7,13,1,5,0,15,10,3,9,8,6],
     [4,2,1,11,10,13,7,8,15,9,12,5,6,3,0,14],
     [11,8,12,7,1,14,2,13,6,15,0,9,10,4,5,3]],

    [[12,1,10,15,9,2,6,8,0,13,3,4,14,7,5,11],
     [10,15,4,2,7,12,9,5,6,1,13,14,0,11,3,8],
     [9,14,15,5,2,8,12,3,7,0,4,10,1,13,11,6],
     [4,3,2,12,9,5,15,10,11,14,1,7,6,0,8,13]],

    [[4,11,2,14,15,0,8,13,3,12,9,7,5,10,6,1],
     [13,0,11,7,4,9,1,10,14,3,5,12,2,15,8,6],
     [1,4,11,13,12,3,7,14,10,15,6,8,0,5,9,2],
     [6,11,13,8,1,4,10,7,9,5,0,15,14,2,3,12]],

    [[13,2,8,4,6,15,11,1,10,9,3,14,5,0,12,7],
     [1,15,13,8,10,3,7,4,12,5,6,11,0,14,9,2],
     [7,11,4,1,9,12,14,2,0,6,10,13,15,3,5,8],
     [2,1,14,7,4,10,8,13,15,12,9,0,3,5,6,11]]]

def generate_round_keys(key_hex):
    key = hex2bin(key_hex)
    left, right = key[:28], key[28:]
    round_keys = []
    for _ in range(1, 17):
        left, right = shift_left(left, 1), shift_left(right, 1)
        round_key = (left + right)[:48]
        round_keys.append(round_key)
    return round_keys

def feistel(right, round_key):
    right_expanded=permute(right,exp_d)
    xored=xor(right_expanded,round_key)
    sbox_out=''
    for i in range(8):
        six=xored[i*6:(i+1)*6]
        row = int(six[0] + six[5], 2)
        col = int(six[1:5], 2)
        sbox_out += format(sbox[i][row][col], '04b')
    return sbox_out

def encrypt(pt_hex,round_keys):
    pt=permute(hex2bin(pt_hex),initial_perm)
    left,right=pt[:32],pt[32:]
    for i in range(16):
        new_right=xor(left,feistel(right,round_keys[i]))
        left,right=right,new_right
    combined=right+left
    return bin2hex(permute(combined,final_perm))

def decrypt(ct_hex, round_keys):
    # Just reverse the round keys
    return encrypt(ct_hex, round_keys[::-1])

# Example usage
plaintext = "ABCDEF0123456789"
key = "133457799BBCDFF1"
round_keys = generate_round_keys(key)

ciphertext = encrypt(plaintext, round_keys)
print("Ciphertext:", ciphertext)

decrypted_text = decrypt(ciphertext, round_keys)
print("Decrypted Text:", decrypted_text)


Ciphertext: A392595A5AE24CD1
Decrypted Text: ABCDEF0123456789


AES

In [ ]:
s_box = [
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
]

inv_s_box = [0]*256
for i, val in enumerate(s_box):
    inv_s_box[val] = i

rcon = [
    0x00,0x01,0x02,0x04,0x08,0x10,0x20,0x40,0x80,0x1b,0x36
]

def gmul(a, b):
    p = 0
    for _ in range(8):
        if b & 1:
            p ^= a
        hi_bit = a & 0x80
        a <<= 1
        if hi_bit:
            a ^= 0x11b
        b >>= 1
    return p % 256

def key_expansion(key):
    key_symbols = [ord(c) for c in key]
    key_schedule = [[] for i in range(44)]
    for i in range(4):
        key_schedule[i] = key_symbols[4*i:4*(i+1)]
    for i in range(4, 44):
        temp = key_schedule[i-1][:]
        if i % 4 == 0:
            temp = temp[1:] + temp[:1]  # rotate
            temp = [s_box[b] for b in temp]
        key_schedule[i] = temp
    return key_schedule

def add_round_key(state, key_schedule, rnd):
    for r in range(4):
        for c in range(4):
            state[r][c] ^= key_schedule[4*rnd+c][r]

def sub_bytes(state):
    for r in range(4):
        for c in range(4):
            state[r][c] = s_box[state[r][c]]

def inv_sub_bytes(state):
    for r in range(4):
        for c in range(4):
            state[r][c] = inv_s_box[state[r][c]]

def shift_rows(state):
    state[1] = state[1][1:] + state[1][:1]
    state[2] = state[2][2:] + state[2][:2]
    state[3] = state[3][3:] + state[3][:3]

def inv_shift_rows(state):
    state[1] = state[1][-1:] + state[1][:-1]
    state[2] = state[2][-2:] + state[2][:-2]
    state[3] = state[3][-3:] + state[3][:-3]

def mix_columns(state):
    for c in range(4):
        a = state[0][c]
        b = state[1][c]
        d = state[2][c]
        e = state[3][c]
        state[0][c] = gmul(a,2) ^ gmul(b,3) ^ d ^ e
        state[1][c] = a ^ gmul(b,2) ^ gmul(d,3) ^ e
        state[2][c] = a ^ b ^ gmul(d,2) ^ gmul(e,3)
        state[3][c] = gmul(a,3) ^ b ^ d ^ gmul(e,2)

def inv_mix_columns(state):
    for c in range(4):
        a = state[0][c]
        b = state[1][c]
        d = state[2][c]
        e = state[3][c]
        state[0][c] = gmul(a,14) ^ gmul(b,11) ^ gmul(d,13) ^ gmul(e,9)
        state[1][c] = gmul(a,9) ^ gmul(b,14) ^ gmul(d,11) ^ gmul(e,13)
        state[2][c] = gmul(a,13) ^ gmul(b,9) ^ gmul(d,14) ^ gmul(e,11)
        state[3][c] = gmul(a,11) ^ gmul(b,13) ^ gmul(d,9) ^ gmul(e,14)
def encrypt_block(plaintext, key_schedule):
    state = [[0]*4 for _ in range(4)]
    for i in range(16):
        state[i%4][i//4] = ord(plaintext[i])
    add_round_key(state, key_schedule, 0)
    for rnd in range(1,10):
        sub_bytes(state)
        shift_rows(state)
        mix_columns(state)
        add_round_key(state, key_schedule, rnd)
    sub_bytes(state)
    shift_rows(state)
    add_round_key(state, key_schedule, 10)
    return ''.join(chr(state[r][c]) for c in range(4) for r in range(4))
def decrypt_block(ciphertext, key_schedule):
    state = [[0]*4 for _ in range(4)]
    for i in range(16):
        state[i%4][i//4] = ord(ciphertext[i])
    add_round_key(state, key_schedule, 10)
    for rnd in range(9,0,-1):
        inv_shift_rows(state)
        inv_sub_bytes(state)
        add_round_key(state, key_schedule, rnd)
        inv_mix_columns(state)
    inv_shift_rows(state)
    inv_sub_bytes(state)
    add_round_key(state, key_schedule, 0)
    return ''.join(chr(state[r][c]) for c in range(4) for r in range(4))
if __name__ == "__main__":
    key = "Thats my AES key"
    plaintext ="ABCDEF0123456789"

    ks = key_expansion(key)
    cipher = encrypt_block(plaintext, ks)
    print("Ciphertext:", [ord(x) for x in cipher])

    decrypted = decrypt_block(cipher, ks)
    print("Decrypted:", decrypted)


Ciphertext: [211, 46, 163, 185, 45, 79, 0, 206, 176, 187, 198, 215, 17, 83, 200, 189]
Decrypted: ABCDEF0123456789


In [ ]:
SBOX = [
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
]
INV_SBOX = [0]*256
for i in range(256):
    INV_SBOX[SBOX[i]] = i
def sub_bytes(s):
    for i in range(16):
        s[i] = SBOX[s[i]]
def inv_sub_bytes(s):
    for i in range(16):
        s[i] = INV_SBOX[s[i]]
def shift_rows(s):
    return [
        s[0],  s[5],  s[10], s[15],
        s[4],  s[9],  s[14], s[3],
        s[8],  s[13], s[2],  s[7],
        s[12], s[1],  s[6],  s[11]
    ]
def inv_shift_rows(s):
    return [
        s[0],  s[13], s[10], s[7],
        s[4],  s[1],  s[14], s[11],
        s[8],  s[5],  s[2],  s[15],
        s[12], s[9],  s[6],  s[3]
    ]
def mix_columns(s):
    for i in range(16):
        s[i] ^= 0xAA
def inv_mix_columns(s):
    for i in range(16):
        s[i] ^= 0xAA
def add_round_key(s, key):
    for i in range(16):
        s[i] ^= key[i]
def aes_encrypt(plaintext, key):
    s = list(plaintext.encode()[:16].ljust(16, b'\x00'))
    k = list(key.encode()[:16].ljust(16, b'\x00'))
    add_round_key(s, k)
    for _ in range(9):
        sub_bytes(s)
        s = shift_rows(s)
        mix_columns(s)
        add_round_key(s, k)
    sub_bytes(s)
    s = shift_rows(s)
    add_round_key(s, k)
    return ''.join(format(x, '02x') for x in s)
def aes_decrypt(cipher_hex, key):
    s = list(bytes.fromhex(cipher_hex))
    k = list(key.encode()[:16].ljust(16, b'\x00'))
    add_round_key(s, k)
    for _ in range(9):
        s = inv_shift_rows(s)
        inv_sub_bytes(s)
        add_round_key(s, k)
        inv_mix_columns(s)
    s = inv_shift_rows(s)
    inv_sub_bytes(s)
    add_round_key(s, k)
    return bytes(s).rstrip(b'\x00').decode(errors='ignore')
pt = input("Enter plaintext: ")
key = input("Enter key: ")
ct = aes_encrypt(pt, key)
print("Encrypted:", ct)
dt = aes_decrypt(ct, key)
print("Decrypted:", dt)

Enter plaintext: fgkjbvbn
Enter key: dfcgvhbjk;
Encrypted: f328efca22adc80fdb4f0f1fa802a09d
Decrypted: fgkjbvbn


SHA

In [ ]:
def rtr(m,n):
    return ((m>>n)^(m<<(32-n))) & 0xFFFFFFFF
H = [ 0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a, 0x510e527f, 0x9b05688c, 0x1f83d9ab,
 0x5be0cd19,
]
def sha(m):
    m=m.encode()
    m+=b'\x80'
    while len(m)%512!=448:
        m+=b'\x00'
    chunks=[m[i:i+64] for i in range(0,len(m),64)]
    for ch in chunks:
        w=list(ch)
        for i in range(64):
            a,b,c,d,e,f,g,h=H[:]
            h=g
            g=f
            f=e
            e=d
            d=c
            c=b
            b=a
            a=h^rtr(a,4)& 0xFFFFFFFF
            H[0]=(H[0]+a) & 0xFFFFFFFF
            H[1]=(H[1]+a) & 0xFFFFFFFF
            H[2]=(H[2]+a) & 0xFFFFFFFF
            H[3]=(H[3]+a) & 0xFFFFFFFF
            H[4]=(H[4]+a) & 0xFFFFFFFF
            H[5]=(H[5]+a) & 0xFFFFFFFF
            H[6]=(H[6]+a) & 0xFFFFFFFF
            H[7]=(H[7]+a) & 0xFFFFFFFF
    s=''
    for v in H:
        s+=f'{v:08x}'
    return s
print(sha(input('enter')))

enterfgjhk8uijk
554968f4a6a7311227ae75ff908f77c73c4dd50c8644eb190ac35c3847204fa6


DSS


In [ ]:
import random
def mod_inverse(a, m):
    return pow(a, -1, m)
def hash_message(msg, q):
    return sum((ord(c) * (i + 1) for i, c in enumerate(msg))) % q
def dss_sign(message, p, q, g, private_key):
    h = hash_message(message, q)
    while True:
        k = random.randint(1, q - 1)
        r = pow(g, k, p) % q
        if r == 0:
            continue
        k_inv = mod_inverse(k, q)
        s = (k_inv * (h + private_key * r)) % q
        if s == 0:
            continue
        return s, r
def dss_verify(message, s, r, p, q, g, public_key):
    if not (0 < r < q and 0 < s < q):
        return False
    h = hash_message(message, q)
    w = mod_inverse(s, q)
    u1 = (h * w) % q
    u2 = (r * w) % q
    v = (pow(g, u1, p) * pow(public_key, u2, p) % p) % q
    return v == r
if __name__ == "__main__":
    q = 11
    p = 23
    g = 4
    private_key = 6
    public_key = pow(g, private_key, p)
    message = "Sriya012"
    s, r = dss_sign(message, p, q, g, private_key)
    print(f"Message: {message}")
    print(f"Signature (s, r): ({s}, {r})")

    valid = dss_verify(message, s, r, p, q, g, public_key)
    if valid:
        print("Signature is VERIFIED for the original message.")
    else:
        print("Signature is NOT VERIFIED for the original message.")

    tampered = "Sriya014"
    print(f"Tampered Message:{tampered}")
    valid_tampered = dss_verify(tampered, s, r, p, q, g, public_key)
    if valid_tampered:
        print("Signature is VERIFIED for the tampered message. (Error!)")
    else:
        print("Signature is NOT VERIFIED for the tampered message.")


Message: Sriya012
Signature (s, r): (10, 4)
Signature is VERIFIED for the original message.
Tampered Message:Sriya014
Signature is NOT VERIFIED for the tampered message.


RSA

In [ ]:
import math
p,q=map(int,input('enter p q').split())
n=p*q
phi=(p-1)*(q-1)
for i in range(2,phi):
    if math.gcd(phi,i)==1:
        e=i
        break
d=pow(e,-1,phi)
m=int(input('enter msg'))
c=pow(m,e,n)
print(c)
print(pow(c,d,n))


enter p q619 607
enter msg42144
136794
42144


Diffie Hellman

In [ ]:
import random
q=int(input('prime'))
a=int(input('primitive root'))
xa=random.randint(1,p-1)
xb=random.randint(1,p-1)
ya=pow(a,xa,q)
yb=pow(a,xb,q)
if pow(yb,xa,q)==pow(ya,xb,q):
    print('yes')
else:
    print('No')


prime619
primitive root3
yes
